In [ ]:
# Source: https://sbme-tutorials.github.io/2021/cv/notes/4_week4.html

import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import time

In [ ]:
# Load and display image
IMG_PATH = Path('img3.jpg')
# IMG_PATH = Path('img2.png')
assert IMG_PATH.exists()
img = cv2.imread(str(IMG_PATH))
img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

fig, ax = plt.subplots(1, 1, figsize=(10, 10))
ax.imshow(img)

In [ ]:
import scipy.signal

# Convert to grayscale
grayscale = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
grayscale = grayscale.astype(np.int32)
print(img.shape)
print(grayscale.shape)

# Compute gradient (convolve image)
sobel_filter_x = np.array([[-1, 0, 1],
                           [-2, 0, 2], 
                           [-1, 0, 1]])

sobel_filter_y = np.array([[ 1,  2,  1],
                           [ 0,  0,  0], 
                           [-1, -2, -1]])

fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(20, 10))

grad_x = scipy.signal.convolve2d(grayscale, sobel_filter_x)
ax[0].imshow(abs(grad_x), cmap='gray')

grad_y = scipy.signal.convolve2d(grayscale, sobel_filter_y)
ax[1].imshow(abs(grad_y), cmap='gray')

grad_img = np.sqrt(grad_x**2 + grad_y**2)
grad_img = grad_img > np.quantile(grad_img, 0.9)
grad_img = cv2.erode(grad_img.astype(np.uint8), kernel=np.ones((2, 2), dtype=np.uint8))
ax[2].imshow(grad_img, cmap='gray')

In [ ]:
def degree_to_radian(degree: int) -> float:
    return degree * (np.pi / 180)

def radian_to_degree(radian: float) -> float:
    return radian * ( 180 / np.pi)

# Parameters
height, width = grayscale.shape
diag_len = int(np.sqrt(height**2 + width**2))

n_thetas = 180
theta_step = np.pi / n_thetas 
theta_grid = np.linspace(-np.pi / 2, np.pi / 2, n_thetas)

n_rhos = 180
rho_step = 2 * diag_len / n_rhos
r_grid = np.linspace(-diag_len, diag_len, n_rhos)

print(f"Nb thetas: {n_thetas}")
print(f"Nb rhos: {n_rhos}")

# Binarize grad_img
bin_img = grad_img > np.quantile(grad_img, 0.90)
points = np.where(bin_img > 0)
print("Nb points: ", len(points[0])) 

cos_t = np.cos(theta_grid)
sin_t = np.sin(theta_grid)

tic = time.time()
hough_matrix = np.zeros((n_rhos + 1, n_thetas + 1), dtype=np.uint16)
for row, col in zip(*points):
    for idx_theta in range(len(theta_grid)):
        r = col * cos_t[idx_theta] + row * sin_t[idx_theta] + diag_len
        idx_r = int(r / rho_step)
        hough_matrix[idx_r, idx_theta] += 1
        
print(f"Took {(time.time() - tic):.1f}s")

fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(15, 15))
ax[0][0].imshow(img)

ax[0][1].imshow(bin_img, cmap='gray')

ax[1][0].imshow(hough_matrix, cmap='gray')
ax[1][0].set_xticks(np.linspace(0, n_thetas, 19)) # Every 10 degrees
ax[1][0].set_xticklabels(np.arange(-90, 91, 10))
ax[1][0].set_xlabel("Angle [degree]")
ax[1][0].set_yticks(np.linspace(0, n_rhos, 10)) 
ax[1][0].set_yticklabels(np.linspace(-diag_len, diag_len, 10).astype(int))
ax[1][0].set_ylabel("Distance from origin [pixels]")

hough_matrix_tmp = (hough_matrix - hough_matrix.min()) / (hough_matrix.max() - hough_matrix.min())
hough_matrix_tmp *= 255
hough_matrix_tmp = hough_matrix_tmp.astype(np.uint8)
dst = cv2.equalizeHist(hough_matrix_tmp)
ax[1][1].imshow(dst, cmap='gray')
ax[1][1].set_xticks(np.linspace(0, n_thetas, 19)) # Every 10 degrees
ax[1][1].set_xticklabels(np.arange(-90, 91, 10))
ax[1][1].set_xlabel("Angle [degree]")
ax[1][1].set_yticks(np.linspace(0, n_rhos, 10)) 
ax[1][1].set_yticklabels(np.linspace(-diag_len, diag_len, 10).astype(int))
ax[1][1].set_ylabel("Distance from origin [pixels]")

In [ ]:
# Peak detection
import numpy as np
from scipy.ndimage.filters import maximum_filter
from scipy.ndimage.morphology import generate_binary_structure, binary_erosion

def detect_peaks(image):
    neighborhood = generate_binary_structure(2, 2) # define an 8-connected neighborhood
    local_max = maximum_filter(image, footprint=neighborhood) == image
    background = (image == 0)
    eroded_background = binary_erosion(background, structure=neighborhood, border_value=1)
    detected_peaks = local_max ^ eroded_background
    return detected_peaks

filtered_hough_matrix = hough_matrix_tmp.copy()
filter_background = np.where(filtered_hough_matrix < np.quantile(filtered_hough_matrix, 0.998))
filtered_hough_matrix[filter_background] = 0

peaks = detect_peaks(filtered_hough_matrix)
print(peaks.sum())

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(peaks)

In [ ]:
# Polar to cartesian
def polar2cart(r, theta):
    x = r * np.cos(theta)
    y = r * np.sin(theta)
    return x, y

# Extract maxima
# extrema = np.where(hough_matrix > 0.80 * hough_matrix.max())
extrema = np.where(peaks>0)
print(len(extrema[0]))

fig, ax = plt.subplots(1, 1, figsize=(10,10))
plt.imshow(img)

for row, col in zip(*extrema):
    theta = theta_grid[col]
    r = r_grid[row]
    x0, y0 = polar2cart(r, theta)
    ax.axline((x0, y0), slope=np.tan(theta + np.pi/2))

In [ ]:
# Compare to cv
edges = cv2.Canny(grayscale.astype(np.uint8), 50, 150, apertureSize=3)
lines = cv2.HoughLines(edges, 1, np.pi/360, 150, ).squeeze()

print(len(lines))

fig, ax = plt.subplots(1, 1, figsize=(10,10))
plt.imshow(img)

for rho, theta in lines:
    x0, y0 = polar2cart(rho, theta)
    ax.axline((x0, y0), slope=np.tan(theta + np.pi/2))